In [15]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications.resnet import ResNet50
import pandas as pd
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import PIL
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization, Activation, GlobalAveragePooling2D
from tensorflow.keras import applications, layers, losses, optimizers, Model
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
import keras.backend as K

# Retrain the model on the new dataset

In [16]:
P = '../input/15-after-processed/15_after_processed/'

In [17]:
train_image = tf.keras.utils.image_dataset_from_directory( P+'train_images/',
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(224, 224),  
)

val_image = tf.keras.utils.image_dataset_from_directory(  P+ 'val_images/',
    labels="inferred",
    label_mode="categorical",
    class_names=None, 
    color_mode="rgb",
    batch_size=32,
    image_size=(224, 224),
)


Found 21108 files belonging to 10019 classes.
Found 5414 files belonging to 4190 classes.


### Reminder: Define a function to preprocess the labels in the validation set

In [18]:
num_classes = len(train_image.class_names)

In [19]:
resnet = Sequential()
pretrained_model = applications.ResNet50(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3), 
    pooling='avg',
    classes=num_classes,
    classifier_activation='softmax',
)

for layer in pretrained_model.layers:
    layer.trainable = False

In [20]:
resnet.add(pretrained_model)
# add batch normalization layer
resnet.add(BatchNormalization())
# flatten the output
resnet.add(Flatten())
# add dropout layer
resnet.add(Dropout(0.5))
# add linear layer for reduction
resnet.add(Dense(512, activation='linear'))
# add classification layer
resnet.add(Dense(num_classes, activation='softmax'))

In [21]:
epochs=25

In [22]:
def get_f1(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [23]:
optimizer = keras.optimizers.Adam(learning_rate=0.0001)

In [24]:
resnet.compile(optimizer=optimizer, 
                loss='categorical_crossentropy',
                metrics=[get_f1])

In [25]:
# Define a function to preprocess the labels in the validation set
def preprocess_labels(image, label):
    label = tf.argmax(label, axis=-1)
    label = tf.one_hot(label, num_classes)
    return image, label

# Preprocess the labels in the validation set
val_image = val_image.map(preprocess_labels)

In [26]:
checkpoint_filepath = 'new_resenet.ckpt'



In [ ]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='auto',
    save_best_only=True,
    verbose=1,
    save_freq='epoch')


resnet.fit(train_image, epochs=epochs, validation_data=val_image, callbacks=[model_checkpoint_callback])

Epoch 1/25
659/660 [============================>.] - ETA: 0s - loss: 8.9693 - get_f1: 1.8393e-04
Epoch 1: val_loss improved from inf to 9.29679, saving model to new_resenet.ckpt
660/660 [==============================] - 90s 129ms/step - loss: 8.9685 - get_f1: 1.8365e-04 - val_loss: 9.2968 - val_get_f1: 0.0000e+00
Epoch 2/25
659/660 [============================>.] - ETA: 0s - loss: 7.2954 - get_f1: 0.0186
Epoch 2: val_loss did not improve from 9.29679
660/660 [==============================] - 86s 129ms/step - loss: 7.2943 - get_f1: 0.0185 - val_loss: 9.7958 - val_get_f1: 0.0000e+00
Epoch 3/25
659/660 [============================>.] - ETA: 0s - loss: 5.2150 - get_f1: 0.1045
Epoch 3: val_loss did not improve from 9.29679
660/660 [==============================] - 85s 128ms/step - loss: 5.2142 - get_f1: 0.1047 - val_loss: 10.7859 - val_get_f1: 0.0000e+00
Epoch 4/25
659/660 [============================>.] - ETA: 0s - loss: 3.3417 - get_f1: 0.2775
Epoch 4: val_loss did not improve from